In [ ]:
#!/usr/bin/env python3
"""
Quantum State Tomography of a Bell State
==========================================

This script demonstrates explicit quantum state tomography in a way that is
reminiscent of photonic experiments (e.g. using quarter-waveplates and polarizers)
while allowing execution either on a simulator or on actual hardware.

Key design features:
  1. Unified circuit construction:
     - A single function builds the tomography circuit.
     - The parameter `add_measurements` toggles whether measurement
       operations are appended (used in simulation mode) or not (for hardware mode).
       
  2. Unified compilation & execution:
     - A helper function compiles the circuits appropriately:
         • For simulation: uses Qiskit's transpile().
         • For hardware: uses a preset pass manager (which produces an ISA–compiled circuit).
     - The remaining logic (circuit construction and postprocessing) remains identical.
       
  3. Clear abstraction of observables:
     - SparsePauliOp is a Qiskit object representing a sparse (efficient) representation of a
       Pauli operator. Here it is used to define the observables (e.g. "ZZ", "ZI", "IZ")
       needed to extract two-qubit correlations and marginal expectation values.
       
  4. “ISA compiled” circuits (here named “compiled circuits”) are those processed by the
     preset pass manager to conform with the native instruction set of the hardware backend.
     
Usage:
  Set `use_simulator=True` to run on the AerSimulator (with counts-based processing).
  Set `use_simulator=False` to run on hardware (using the Estimator API).
  
Before running on hardware, ensure your IBM Quantum account is configured.
"""

import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

# Define Pauli matrices for density matrix reconstruction.
I_mat = np.array([[1, 0], [0, 1]])
X_mat = np.array([[0, 1], [1, 0]])
Y_mat = np.array([[0, -1j], [1j, 0]])
Z_mat = np.array([[1, 0], [0, -1]])

def create_phi_plus_state():
    """Prepare the Bell state |φ⁺⟩ = (|00⟩ + |11⟩)/√2."""
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    return qc

def create_tomography_circuit(base_circ, meas_bases, add_measurements=True):
    """
    Build a tomography circuit that:
      - Prepares the state.
      - Applies rotations to change the measurement basis:
            X: H; Y: S† then H; Z: no rotation.
      - Optionally appends measurements in the Z basis.

    Parameters:
      base_circ (QuantumCircuit): The state-preparation circuit.
      meas_bases (list of str): Measurement bases for each qubit (e.g. ['X','Z']).
      add_measurements (bool): If True, append measurement operations (for simulation mode).

    Returns:
      QuantumCircuit: The constructed tomography circuit.
    """
    num_qubits = base_circ.num_qubits
    if add_measurements:
        qr = QuantumRegister(num_qubits)
        cr = ClassicalRegister(num_qubits)
        qc = QuantumCircuit(qr, cr)
        qc.compose(base_circ, qubits=qr, inplace=True)
        for i, basis in enumerate(meas_bases):
            if basis.upper() == 'X':
                qc.h(qr[i])
            elif basis.upper() == 'Y':
                qc.sdg(qr[i])
                qc.h(qr[i])
            # For Z, no rotation is needed.
        for i in range(num_qubits):
            qc.measure(qr[i], cr[i])
        return qc
    else:
        # Build the circuit without measurement operations.
        qc = base_circ.copy()
        for i, basis in enumerate(meas_bases):
            if basis.upper() == 'X':
                qc.h(i)
            elif basis.upper() == 'Y':
                qc.sdg(i)
                qc.h(i)
        return qc

def compile_circuits(circuits, backend, use_hardware):
    """
    Compile a list of circuits for the given backend.
    
    Parameters:
      circuits (list of QuantumCircuit): Circuits to compile.
      backend: Target backend (simulator or hardware).
      use_hardware (bool): True if the target is hardware.

    Returns:
      list of QuantumCircuit: The compiled circuits.
    """
    if use_hardware:
        # Use a preset pass manager to produce an ISA–compiled circuit.
        pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
        return [pm.run(circ) for circ in circuits]
    else:
        return transpile(circuits, backend)

def get_observables_for_setting(setting):
    """
    For a given measurement setting (e.g. ('X','Z')), return the list of observables.
    
    The primary observable is "ZZ". Additionally, for settings where a marginal is needed,
    "ZI" (for qubit 0) or "IZ" (for qubit 1) is appended.

    SparsePauliOp is a Qiskit object representing a sparse Pauli operator.
    """
    obs = [SparsePauliOp("ZZ")]
    if setting in [('X', 'Z'), ('Y', 'Z'), ('Z', 'Z')]:
        obs.append(SparsePauliOp("ZI"))
    if setting in [('Z', 'X'), ('Z', 'Y'), ('Z', 'Z')]:
        obs.append(SparsePauliOp("IZ"))
    return obs

def compute_two_qubit_expectation(counts):
    """Compute two-qubit expectation value from counts (0→+1, 1→–1)."""
    total = sum(counts.values())
    exp_val = 0
    for outcome, count in counts.items():
        a = 1 if outcome[0] == '0' else -1
        b = 1 if outcome[1] == '0' else -1
        exp_val += (a * b) * count
    return exp_val / total

def compute_marginal(counts, qubit_index):
    """Compute single-qubit marginal expectation value."""
    total = sum(counts.values())
    exp_val = 0
    for outcome, count in counts.items():
        val = 1 if outcome[qubit_index] == '0' else -1
        exp_val += val * count
    return exp_val / total

def process_simulation_results(circuit_dict, results):
    """
    Process results from the simulator.
    
    Returns a dictionary mapping each setting to its computed expectation values.
    """
    counts_dict = {setting: results.get_counts(circ)
                   for setting, circ in circuit_dict.items()}
    
    # Compute two-qubit correlations and single-qubit marginals.
    E_xx = compute_two_qubit_expectation(counts_dict[('X', 'X')])
    E_xy = compute_two_qubit_expectation(counts_dict[('X', 'Y')])
    E_xz = compute_two_qubit_expectation(counts_dict[('X', 'Z')])
    E_yx = compute_two_qubit_expectation(counts_dict[('Y', 'X')])
    E_yy = compute_two_qubit_expectation(counts_dict[('Y', 'Y')])
    E_yz = compute_two_qubit_expectation(counts_dict[('Y', 'Z')])
    E_zx = compute_two_qubit_expectation(counts_dict[('Z', 'X')])
    E_zy = compute_two_qubit_expectation(counts_dict[('Z', 'Y')])
    E_zz = compute_two_qubit_expectation(counts_dict[('Z', 'Z')])
    
    E_xI = compute_marginal(counts_dict[('X', 'Z')], 0)
    E_yI = compute_marginal(counts_dict[('Y', 'Z')], 0)
    E_zI = compute_marginal(counts_dict[('Z', 'Z')], 0)
    E_Ix = compute_marginal(counts_dict[('Z', 'X')], 1)
    E_Iy = compute_marginal(counts_dict[('Z', 'Y')], 1)
    E_Iz = compute_marginal(counts_dict[('Z', 'Z')], 1)
    
    return {
        ('I', 'I'): 1,
        ('X', 'I'): E_xI,
        ('Y', 'I'): E_yI,
        ('Z', 'I'): E_zI,
        ('I', 'X'): E_Ix,
        ('I', 'Y'): E_Iy,
        ('I', 'Z'): E_Iz,
        ('X', 'X'): E_xx,
        ('X', 'Y'): E_xy,
        ('X', 'Z'): E_xz,
        ('Y', 'X'): E_yx,
        ('Y', 'Y'): E_yy,
        ('Y', 'Z'): E_yz,
        ('Z', 'X'): E_zx,
        ('Z', 'Y'): E_zy,
        ('Z', 'Z'): E_zz
    }

def process_hardware_results(task_keys, results):
    """
    Process results from hardware execution via the Estimator API.
    
    Each PubResult in results contains a list of expectation values in its
    data.evs attribute.
    """
    expvals = {}
    for i, setting in enumerate(task_keys):
        vals = results[i].data.evs  # List of expectation values.
        expvals[setting] = vals[0].real
        if setting in [('X', 'Z'), ('Y', 'Z'), ('Z', 'Z')] and len(vals) >= 2:
            expvals[(setting, "q0")] = vals[1].real
        if setting in [('Z', 'X'), ('Z', 'Y'), ('Z', 'Z')] and len(vals) >= 3:
            expvals[(setting, "q1")] = vals[2].real

    # Unify into a single dictionary.
    return {
        ('I', 'I'): 1,
        ('X', 'I'): expvals.get((('X', 'Z'), "q0")),
        ('Y', 'I'): expvals.get((('Y', 'Z'), "q0")),
        ('Z', 'I'): expvals.get((('Z', 'Z'), "q0")),
        ('I', 'X'): expvals.get((('Z', 'X'), "q1")),
        ('I', 'Y'): expvals.get((('Z', 'Y'), "q1")),
        ('I', 'Z'): expvals.get((('Z', 'Z'), "q1")),
        ('X', 'X'): expvals.get(('X', 'X')),
        ('X', 'Y'): expvals.get(('X', 'Y')),
        ('X', 'Z'): expvals.get(('X', 'Z')),
        ('Y', 'X'): expvals.get(('Y', 'X')),
        ('Y', 'Y'): expvals.get(('Y', 'Y')),
        ('Y', 'Z'): expvals.get(('Y', 'Z')),
        ('Z', 'X'): expvals.get(('Z', 'X')),
        ('Z', 'Y'): expvals.get(('Z', 'Y')),
        ('Z', 'Z'): expvals.get(('Z', 'Z'))
    }

def run_tomography(use_simulator=True, shots=8192, backend_name=None):
    """
    Run the tomography routine on either a simulator or on hardware.
    
    Parameters:
      use_simulator (bool): If True, run on AerSimulator (with measurements and counts);
                            if False, run on hardware using the Estimator API.
      shots (int): Number of shots per circuit (or per task in hardware mode).
      backend_name (str, optional): IBM Quantum backend name (default: 'ibm_kyiv' for hardware).
    """
    base_circ = create_phi_plus_state()
    bases = ['Z', 'X', 'Y']
    
    # Build a dictionary of tomography circuits for all measurement settings.
    # The same function is used for both simulation and hardware modes;
    # the add_measurements flag toggles whether measurement operations are appended.
    circuit_dict = {}
    for b0 in bases:
        for b1 in bases:
            setting = (b0, b1)
            circuit = create_tomography_circuit(base_circ, [b0, b1], add_measurements=use_simulator)
            circuit_dict[setting] = circuit

    # Select backend and compile circuits.
    if use_simulator:
        backend = AerSimulator()
    else:
        try:
            service = QiskitRuntimeService(instance='ibm-q/open/main')
        except Exception as e:
            print("Error loading IBM Quantum account:", e)
            return
        if backend_name is None:
            backend_name = 'ibm_kyiv'
        backend = service.backend(backend_name)
    
    compiled_circuits = compile_circuits(list(circuit_dict.values()), backend, use_hardware=not use_simulator)
    
    if use_simulator:
        print("Running on simulator:", backend.name)
        job = backend.run(compiled_circuits, shots=shots)
        result = job.result()
        expvals = process_simulation_results(circuit_dict, result)
    else:
        print("Running on hardware backend:", backend.name)
        # For hardware, prepare a task for each compiled circuit.
        tasks = []
        task_keys = []
        for setting, circ in zip(circuit_dict.keys(), compiled_circuits):
            obs_list = get_observables_for_setting(setting)
            # Map observables to the circuit layout.
            mapped_obs = [obs.apply_layout(circ.layout) for obs in obs_list]
            tasks.append((circ, mapped_obs))
            task_keys.append(setting)
        
        estimator = Estimator(mode=backend)
        estimator.options.resilience_level = 1
        estimator.options.default_shots = shots
        job = estimator.run(tasks)
        print("Job ID:", job.job_id())
        results = job.result()
        expvals = process_hardware_results(task_keys, results)
    
    # Reconstruct the density matrix via linear inversion:
    #   ρ = 1/4 ∑₍ₐ,ₑ₎ E(a⊗b) (a ⊗ b)
    pauli_dict = {'I': I_mat, 'X': X_mat, 'Y': Y_mat, 'Z': Z_mat}
    rho = np.zeros((4, 4), dtype=complex)
    for (a, b), val in expvals.items():
        if val is not None:
            rho += val * np.kron(pauli_dict[a], pauli_dict[b])
    rho /= 4.0
    
    print("\nReconstructed density matrix:")
    np.set_printoptions(precision=3, suppress=True)
    print(rho)
    
    # Visualization.
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    im0 = axs[0].imshow(np.real(rho), cmap='viridis', interpolation='none')
    axs[0].set_title("Real part of ρ")
    for i in range(4):
        for j in range(4):
            axs[0].text(j, i, f"{np.real(rho)[i, j]:.2f}", ha='center', va='center', color='w')
    plt.colorbar(im0, ax=axs[0])
    
    im1 = axs[1].imshow(np.imag(rho), cmap='viridis', interpolation='none')
    axs[1].set_title("Imaginary part of ρ")
    for i in range(4):
        for j in range(4):
            axs[1].text(j, i, f"{np.imag(rho)[i, j]:.2f}", ha='center', va='center', color='w')
    plt.colorbar(im1, ax=axs[1])
    
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    # Set use_simulator=True for simulation mode; use_simulator=False for hardware mode.
    run_tomography(use_simulator=False, shots=64)
